[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/intel/openfl/blob/develop/openfl-tutorials/interactive_api/numpy_linear_regression/workspace/SingleNotebook.ipynb)

### Preparations in colab:
We need to clone the repository to run a federation because it contains director and envoy configs to start from.

1. Clone the OpenFL repository
2. Install OpenFL 
3. Go to the linreg workspace

In [ ]:
# For right now, install from source, later we would migrate to PyPI install
# !pip install openfl==1.2.1
import shutil
#shutil.rmtree('./openfl')
!git clone https://github.com/walteriviera/openfl.git
!cd openfl && git checkout develop && pip install . --upgrade pip
!pip install medmnist

Cloning into 'openfl'...
remote: Enumerating objects: 4024, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 4024 (delta 149), reused 197 (delta 92), pack-reused 3723
Receiving objects: 100% (4024/4024), 3.20 MiB | 11.00 MiB/s, done.
Resolving deltas: 100% (2274/2274), done.
Branch 'develop' set up to track remote branch 'develop' from 'origin'.
Switched to a new branch 'develop'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/openfl
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |███████████████████

In [ ]:
!pip install acsconv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for acsconv: filename=ACSConv-0.1.1-py3-none-any.whl size=24202 sha256=926a835f39a53c3ff4469fbae29b6b7745bc23727161106b097124e5ecddd152
  Stored in directory: /root/.cache/pip/wheels/3a/91/f3/4acda60ba5f1c198dee71b6537a78e202f204a97bb75547cc3
Successfully built acsconv


In [ ]:
import os
from time import sleep
os.chdir('openfl/openfl-tutorials/interactive_api/PyTorch_MedMNIST_3D/workspace')

In [ ]:
# Install dependencies if not already installed
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torch.nn.functional as F

import medmnist

from medmnist import INFO, Evaluator

# Model definition

In [ ]:
num_epochs = 3
TRAIN_BS, VALID_BS = 64, 128

lr = 0.001
gamma=0.1
milestones = [0.5 * num_epochs, 0.75 * num_epochs]
data_flag = 'synapsemnist3d'

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

In [ ]:
# define a simple CNN model
class Net(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Sequential(
            nn.Linear(64 * 4 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes))

    def forward(self, x):
        x = self.layer1(x)
        #print(f"L1_out: {x.shape}")
        x = self.layer2(x)
        #print(f"L2_out: {x.shape}")
        x = self.layer3(x)
        #print(f"L3_out: {x.shape}")
        x = self.layer4(x)
        #print(f"L4_out: {x.shape}")
        x = self.layer5(x)
        #print(f"L5_out: {x.shape}")
        x = x.view(x.size(0), -1)
        #print(f"x.view(x.size(0)_out: {x.shape}")
        x = self.fc(x)
        return x

In [ ]:
from wspace_utils.utils import Transform3D, model_to_syncbn
# define a simple CNN model

'''
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
               
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, in_channels=1, num_classes=2):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(in_channels, num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], in_channels=in_channels, num_classes=num_classes)


def ResNet50(in_channels, num_classes):
    return ResNet(Bottleneck, [3, 4, 6, 3], in_channels=in_channels, num_classes=num_classes)
'''



'\nclass BasicBlock(nn.Module):\n    expansion = 1\n\n    def __init__(self, in_planes, planes, stride=1):\n        super(BasicBlock, self).__init__()\n        self.conv1 = nn.Conv2d(\n            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)\n        self.bn1 = nn.BatchNorm2d(planes)\n        \n        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,\n                               stride=1, padding=1, bias=False)\n        self.bn2 = nn.BatchNorm2d(planes)\n\n        self.shortcut = nn.Sequential()\n        if stride != 1 or in_planes != self.expansion*planes:\n            self.shortcut = nn.Sequential(\n                nn.Conv2d(in_planes, self.expansion*planes,\n                          kernel_size=1, stride=stride, bias=False),\n                nn.BatchNorm2d(self.expansion*planes)\n               \n            )\n\n    def forward(self, x):\n        out = F.relu(self.bn1(self.conv1(x)))\n        out = self.bn2(self.conv2(out))\n        out += self.s

In [ ]:
from acsconv.converters import ACSConverter, Conv3dConverter, Conv2_5dConverter

#model = ResNet18(in_channels=n_channels, num_classes=n_classes)
model = Net(in_channels=n_channels, num_classes=n_classes)

model = model_to_syncbn(Conv3dConverter(model, i3d_repeat_axis=None))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# from torchvision import models
print(model)

The ``converters`` are currently experimental. It may not support operations including (but not limited to) Functions in ``torch.nn.functional`` that involved data dimension
Conv3dConverter(
Net(
  (layer1): Sequential(
    (0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): SynchronizedBatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): SynchronizedBatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv3d(16, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): SynchronizedBatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer4): Sequential(
    (0): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  

# Set-up Federation

## 1. Start the Director service and several envoys with generated data

In [ ]:
# Here are the main parameters for our Federation
n_cols = 2
datapath = 'data/.' 
dataname = 'synapsemnist3d'

In [ ]:
import os
from pathlib import Path
import yaml
from typing import Dict, List, Union

## Start the Director service

In [ ]:
cwd = Path.cwd()
director_workspace_path = Path('../director/').absolute()
director_config_file = director_workspace_path / 'director_config.yaml'
director_logfile = director_workspace_path / 'director.log'
if director_logfile.is_file(): director_logfile.unlink()

os.environ['main_folder'] = str(cwd)
os.environ['director_workspace_path'] = str(director_workspace_path)
os.environ['director_logfile'] = str(director_logfile)
os.environ['director_config_file'] = str(director_config_file)

In [ ]:
%%script /bin/bash --bg
cd $director_workspace_path
fx director start --disable-tls -c $director_config_file > $director_logfile &
cd $main_folder

Starting job # 0 in a separate thread.


## Start Envoys

#### First, we create several envoy config files 

In [ ]:
# Read the original envoy config file content
with open(Path('../envoy/envoy_config.yaml'), "r") as stream:
    orig_config = yaml.safe_load(stream)

def generate_envoy_configs(config: Dict,
                           save_path: Union[str, Path] = '../envoy/',
                           n_cols: int = 2,
                           datapath: str = '',
                           dataname: str = 'bloodmnist') -> List[Path]:

    config['shard_descriptor']['params']['datapath'] = datapath
    config['shard_descriptor']['params']['dataname'] = dataname
        
    config_paths = [(Path(save_path) / f'{i}_envoy_config.yaml').absolute()
                for i in range(1, n_cols + 1)]

    for i, path in enumerate(config_paths):
        config['shard_descriptor']['params']['rank_worldsize'] = str(i+1) + ", " + str(n_cols)
        with open(path, "w") as stream:
            yaml.safe_dump(config, stream)
            
    return config_paths
            
def remove_configs(config_paths):
    for path in config_paths:
        path.unlink()

In [ ]:
config_paths = generate_envoy_configs(orig_config,
                                      n_cols=n_cols,
                                      datapath=datapath,
                                      dataname=dataname)

#### Now start Envoy processes in a loop

In [ ]:
# envoy_workspace_path = Path('../envoy/').absolute()
def start_envoys(config_paths: List[Path]) -> None:
    envoy_workspace_path = config_paths[0].parent
    cwd = Path.cwd()
    os.chdir(envoy_workspace_path)
    for i, path in enumerate(config_paths):
        os.system(f'fx envoy start -n env_{i + 1} --disable-tls '
                  f'--envoy-config-path {path} -dh localhost -dp 50051 '
                  f'>env_{i + 1}.log &')
    os.chdir(cwd)

sleep(5)

start_envoys(config_paths)

sleep(25)

## 2. Connect to the Director service of out Federation as Data scientist

In [ ]:
# Create a federation
from openfl.interface.interactive_api.federation import Federation

# please use the same identificator that was used in signed certificate
client_id = 'frontend'
director_node_fqdn = 'localhost'
director_port=50051

# 2) Run with TLS disabled (trusted environment)
# Federation can also determine local fqdn automatically
federation = Federation(
    client_id=client_id,
    director_node_fqdn=director_node_fqdn,
    director_port=director_port, 
    tls=False)

In [ ]:
# Data scientist may request a list of connected envoys
shard_registry = federation.get_shard_registry()

# WARNING!

# Make sure shard registry contains all the envoys you started!
# In other case try rereconnecting to the Director (the cell above).
shard_registry

{'env_1': {'current_time': '2022-07-01 14:29:18',
  'experiment_name': 'ExperimentName Mock',
  'is_experiment_running': False,
  'is_online': True,
  'last_updated': '2022-07-01 14:28:53',
  'shard_info': node_info {
    name: "env_1"
  }
  shard_description: "MedMNIST dataset, shard number 1 out of 2"
  sample_shape: "28"
  sample_shape: "28"
  sample_shape: "28"
  target_shape: "1"
  target_shape: "1",
  'valid_duration': seconds: 120},
 'env_2': {'current_time': '2022-07-01 14:29:18',
  'experiment_name': 'ExperimentName Mock',
  'is_experiment_running': False,
  'is_online': True,
  'last_updated': '2022-07-01 14:28:53',
  'shard_info': node_info {
    name: "env_2"
  }
  shard_description: "MedMNIST dataset, shard number 2 out of 2"
  sample_shape: "28"
  sample_shape: "28"
  sample_shape: "28"
  target_shape: "1"
  target_shape: "1",
  'valid_duration': seconds: 120}}

### Now we will prepare an FL experiment using OpenFL Python API

*   List item
*   List item



#Data section

#### Data loading
#### Data preprocessing
#### Data injestion

In [ ]:
from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface, FLExperiment
from PIL import Image

shape_transform = False

train_transform = Transform3D(mul='random') if shape_transform else Transform3D()
eval_transform = Transform3D(mul='0.5') if shape_transform else Transform3D()

In [ ]:
class TransformedDataset(Dataset):
    """Image Person ReID Dataset."""

    def __init__(self, dataset, transform=None, target_transform=None, as_rgb=False):
        """Initialize Dataset."""
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform
        self.as_rgb = as_rgb

    def __len__(self):
        """Length of dataset."""
        return len(self.dataset)

    def __getitem__(self, index):
                
        img, label = self.dataset[index]
        
        if self.target_transform:
            label = self.target_transform(label)  
        else:
            label = label.astype(int)
        
        img = np.stack([img/255.]*(3 if self.as_rgb else 1), axis=0)       
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, label



In [ ]:
class MedMnistFedDataset(DataInterface):
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        
    @property
    def shard_descriptor(self):
        return self._shard_descriptor
        
    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor  will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor

        self.train_set = TransformedDataset(
            self._shard_descriptor.get_dataset('train'),
            transform=train_transform, as_rgb=False
        )       
        
        self.valid_set = TransformedDataset(
            self._shard_descriptor.get_dataset('val'),
            transform=eval_transform, as_rgb=False
        )
        
    def get_train_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        return DataLoader(
            self.train_set, num_workers=8, batch_size=self.kwargs['train_bs'], shuffle=True)

    def get_valid_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks without optimizer in contract
        """
        return DataLoader(self.valid_set, num_workers=8, batch_size=self.kwargs['valid_bs'])

    def get_train_data_size(self):
        """
        Information for aggregation
        """
        return len(self.train_set)

    def get_valid_data_size(self):
        """
        Information for aggregation
        """
        return len(self.valid_set)
    



In [ ]:
fed_dataset = MedMnistFedDataset(train_bs=TRAIN_BS, valid_bs=VALID_BS)

### Model Interface

In [ ]:
from copy import deepcopy

framework_adapter = 'openfl.plugins.frameworks_adapters.pytorch_adapter.FrameworkAdapterPlugin'
MI = ModelInterface(model=model.model, optimizer=optimizer, framework_plugin=framework_adapter)

# Save the initial model state
initial_model = deepcopy(model.model)

In [ ]:
TI = TaskInterface()

import torch
import tqdm

train_custom_params={'criterion':criterion,'task':task}

# Task interface currently supports only standalone functions.
@TI.add_kwargs(**train_custom_params)
@TI.register_fl_task(model='model', data_loader='train_loader',
                     device='device', optimizer='optimizer')
def train(model, train_loader, device, optimizer, criterion, task):
    total_loss = []
    
    train_loader = tqdm.tqdm(train_loader, desc="train")
    model.train()
    model.to(device)
    
    for inputs, targets in train_loader:
    
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32).to(device)
            loss = criterion(outputs, targets)
        else:
            targets = torch.squeeze(targets, 1).long().to(device)
            loss = criterion(outputs, targets)
        
        total_loss.append(loss.item())
        
        loss.backward()
        optimizer.step()

    return {'train_loss': np.mean(total_loss),}


val_custom_params={'criterion':criterion, 
                   'run':'model1',
                   'task':task}

@TI.add_kwargs(**val_custom_params)
@TI.register_fl_task(model='model', data_loader='val_loader', device='device')
def validate(model, val_loader, device, criterion, run, task):

    val_loader = tqdm.tqdm(val_loader, desc="validate")
    model.eval()
    model.to(device)

    val_score = 0
    total_samples = 0
    total_loss = []
    y_score = torch.tensor([]).to(device)

    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs.to(device))
            
            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32).to(device)
                loss = criterion(outputs, targets)
                m = nn.Sigmoid()
                outputs = m(outputs).to(device)
            else:
                targets = torch.squeeze(targets, 1).long().to(device)
                loss = criterion(outputs, targets)
                m = nn.Softmax(dim=1)
                outputs = m(outputs).to(device)
                targets = targets.float().resize_(len(targets), 1)

            total_loss.append(loss.item())
            
            total_samples += targets.shape[0]
            pred = outputs.argmax(dim=1)
            val_score += pred.eq(targets).sum().cpu().numpy()
        
        acc = val_score / total_samples        
        test_loss = sum(total_loss) / len(total_loss)

        return {'acc': acc,
                'test_loss': test_loss,
                }


### Run

In [ ]:
experiment_name = 'medmnist_exp'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name)

In [ ]:
fl_experiment.start(model_provider=MI, 
                    task_keeper=TI,
                    data_loader=fed_dataset,
                    rounds_to_train=3,
                    device_assignment_policy='CUDA_PREFERRED')

[14:30:31] INFO     Building 🡆 Object CloudpickleSerializer from openfl.plugins.interface_serializer.cloudpickle_serializer Module.                  ]8;id=791839;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=836850;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py#173\173]8;;\

           INFO     Building 🡆 Object FrameworkAdapterPlugin from openfl.plugins.frameworks_adapters.pytorch_adapter Module.                         ]8;id=312399;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=631228;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py#173\173]8;;\

[14:30:32] INFO     Starting experiment!                                                                                                       ]8;id=548880;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=136416;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#213\213]8;;\

           INFO     FL-Plan hash is 598ceb5bd18f0236ff9e0eb0274d6b4654589b5ae585aca5accfa500a1d1f71fd799fe1124f3b3a4c9387aa308dc0ba1                 ]8;id=102573;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=124120;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py#234\234]8;;\

           INFO     FL-Plan hash is 598ceb5bd18f0236ff9e0eb0274d6b4654589b5ae585aca5accfa500a1d1f71fd799fe1124f3b3a4c9387aa308dc0ba1                 ]8;id=470861;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=951229;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py#234\234]8;;\

...

           INFO     Building 🡆 Object FrameworkAdapterPlugin from openfl.plugins.frameworks_adapters.pytorch_adapter Module.                         ]8;id=591238;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=619264;file:///usr/local/lib/python3.7/dist-packages/openfl/federated/plan/plan.py#173\173]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=766701;file:///usr/local/lib/python3.7/dist-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=545385;file:///usr/local/lib/python3.7/dist-packages/openfl/utilities/utils.py#170\170]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=882007;file:///usr/local/lib/python3.7/dist-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=225498;file:///usr/local/lib/python3.7/dist-packages/openfl/utilities/utils.py#170\170]8;;\

           INFO     SetNewExperiment                                                                                                      ]8;id=750889;file:///usr/local/lib/python3.7/dist-packages/openfl/transport/grpc/director_client.py\director_client.py]8;;\:]8;id=798830;file:///usr/local/lib/python3.7/dist-packages/openfl/transport/grpc/director_client.py#203\203]8;;\

           INFO     Experiment was accepted and launched.                                                                                      ]8;id=778989;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=792042;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#227\227]8;;\

In [ ]:
# This method not only prints messages recieved from the director, 
# but also saves logs in the tensorboard format (by default)
fl_experiment.stream_metrics()

[14:33:13] METRIC   Round 0, collaborator trained train result train_loss:  0.700226                                                           ]8;id=203206;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=874257;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

[14:33:15] METRIC   Round 0, collaborator trained train result train_loss:  0.772596                                                           ]8;id=615171;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=216254;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

[14:33:22] METRIC   Round 0, collaborator validate_local locally_tuned_model_validate result acc:   13.090909                                  ]8;id=14559;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=844760;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

           METRIC   Round 0, collaborator validate_local locally_tuned_model_validate result test_loss:     0.694484                           ]8;id=307895;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=355913;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

[14:33:23] METRIC   Round 0, collaborator validate_local locally_tuned_model_validate result acc:   93.363640                                  ]8;id=757809;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=125007;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

...

[14:35:46] METRIC   Round 2, collaborator Aggregator train result train_loss:       0.561912                                                   ]8;id=918354;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=309100;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

           METRIC   Round 2, collaborator Aggregator locally_tuned_model_validate result acc:       39.670456                                  ]8;id=521036;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=987090;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

           METRIC   Round 2, collaborator Aggregator locally_tuned_model_validate result test_loss: 0.692384                                   ]8;id=937550;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=671763;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

           METRIC   Round 2, collaborator Aggregator aggregated_model_validate result acc:  92.039772                                          ]8;id=494929;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=637558;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

           METRIC   Round 2, collaborator Aggregator aggregated_model_validate result test_loss:    0.693755                                   ]8;id=216353;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=380266;file:///usr/local/lib/python3.7/dist-packages/openfl/interface/interactive_api/experiment.py#116\116]8;;\

## Cleaning

In [ ]:
# To stop all services run
!pkill fx

In [ ]:
remove_configs(config_paths)